In [ ]:
from src.funcs_importar import import_malha_cartog

def cartas(escala,ids):
    '''
    
    '''
    print('# --- Iniciando seleção de área de estudo')
    malha_cartog_gdf_select = import_malha_cartog(escala,ids)
    malha_cartog_gdf_select.set_index('id_folha',inplace=True)
    print(malha_cartog_gdf_select)

    # CRIANDO UM DICIONÁRIO DE CARTAS
    #malha_cartog_df_select = malha_cartog_gdf_select.drop(columns=['geometry']) # Transforming GeoDataFrame  to DataFrame by droping 'geometry' column
    malha_cartog_gdf_select['raw_data'] = ''

    print('# --- Construindo Dicionario de Cartas')
    dic_cartas = malha_cartog_gdf_select.to_dict()

    # MAIS DE UMA FOLHA DE CARTA SELECIONADA
    if len(dic_cartas['raw_data']) > 1:
        print(f"{len(dic_cartas['raw_data'])} folhas cartográfica selecionadas")
        print("")

    # APENAS UMA FOLHA DE CARTA SELECIONADA
    if len(dic_cartas['raw_data']) == 1:
        print(f"{len(dic_cartas['raw_data'])} folha cartográfica selecionada")
        print("")
        
    return dic_cartas, malha_cartog_gdf_select
# ----------------------------------------------------------------------------------------------------------------------

# Automaçao da Malha Cartografica

In [ ]:
from src.funcs_importar import importar # Funçoes descritas no   1 - Geopandas: Creating Geodatabase.ipynb
from src.plots import plot_base    # Funçoes definidas no   1.1 - Geapandas: Manipulating GeoDataFrames.ipynb

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget
from IPython.display import Image

## Com a funçao de produçao de grids da biblioteca GDAL disponibilzada pelo software livre QGIS foi possivel construir uma malha cartografica para as escalas de 1:25.000, 50.000, 100.000, 250.000 e 1.000.000 

## E a partir destas malhas cartograficas vamos setorizar o territorio continental brasileiro nomeando cada setor a partir do sistema de automatizaçao de folhas de cartas que utiliza os vertices da geometria retangular para nomear cada setor.  a biblioteca GeoPandas do primeiro grupo de JupyterNotebooks

In [ ]:
Image('imagens/art_folhas_50k.png')

In [ ]:
# Esta funçao foi desenvolvida em conjunto com o aluno Hilo Goes do curso de Ciencias de Tecnologia da Universidade do Rio Grande do Norte
# Esta funçao apresenta erros para as folhas cartograficas da ZONAS NORTE, nomeando as folhas do hemisferio norte com ordem invertida.[espelhada]
# Para as folhas de 1kk sao descritos 24 zonas porem 24*4 resulta em 92. As coordenadas terminam em 84 Sul e 80 Norte. Sem projeçoes para os polos.

import math
import pandas as pd

# Nomeador de Grids ------------------------------------------------------------------------------------------#
def nomeador_grid(left,right,top,bottom,escala=5):
    e1kk = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'] 
    e500k = [['V','Y'],['X','Z']]
    e250k = [['A','C'],['B','D']]
    e100k = [['I','IV'],['II','V'],['III','VI']]
    e50k = [['1','3'],['2','4']]
    e25k = [['NW','SW'],['NE','SE']]

    if left > right:
        print('Oeste deve ser menor que leste')
    if top < bottom:
        print('Norte deve ser maior que Sul')
        
    # Para corrigir o erro de nomeaçao devemos adicionar um criterio caso a folha pertença ao N

    # id_folha
    # north
    # index
    # numero
    # lat_gap

    else:
        id_folha = ''
        if top <= 0:
            id_folha += 'S'                       # soma a letra S `a string 'id_folha'
            north = False                         # Indexador de folhas de Latitude Positiva
            index = math.floor(-top/4)            # Retorna o maior inteiro menor ou igual a x: {math.floor(x)}
        else:
            id_folha += 'N'                       
            north = True
            index = math.floor(bottom/4)
        
        numero    = math.ceil((180 + right) / 6)
        id_folha += e1kk[index] + str(numero)
        lat_gap   = abs(top - bottom)

        #p500k-----------------------
        if (lat_gap <= 2) & (escala >= 1):
            LO = math.ceil(right / 3) % 2 == 0
            NS = math.ceil(top/2) % 2 != north
            id_folha+='_'+e500k[LO][NS]

        #p250k-----------------------
        if (lat_gap<=1) & (escala>=2):
            LO = math.ceil(right/1.5) % 2 == 0
            NS = math.ceil(top) % 2 != north
            id_folha += e250k[LO][NS]

        #p100k-----------------------
        if (lat_gap <= 0.5) & (escala >= 3):
            LO = (math.ceil(right/0.5) % 3) - 1
            NS = math.ceil(top/0.5) % 2 != north
            id_folha += '_' + e100k[LO][NS]

        #p50k------------------------
        if (lat_gap <= 0.25) & (escala >= 4):
            LO = math.ceil(right/0.25) % 2 == 0
            NS = math.ceil(top/0.25) % 2 != north
            id_folha += e50k[LO][NS]

        #p25k------------------------
        if (lat_gap <= 0.125) & (escala >= 5):
            LO = math.ceil(right/0.125) % 2 == 0
            NS = math.ceil(top/0.125) % 2 != north
            id_folha += '_' + e25k[LO][NS]

        return id_folha

In [ ]:
# Lendo cama vertorial gerada no QGIS com poligonos de 6º x 4º [Folhas em escala de 1:1.000.000]
malha_cartog_1kk = gpd.read_file(gdb+'cartografia/malha_cartog_1kk_wgs84.shp')
malha_cartog_1kk.head(5)

In [ ]:
plots.plot_brazil(malha_cartog_1kk.boundary)
#labels(malha_cartog_1kk)

#### Definindo as regioes delimitadores para cada unidade da malha consturida no qgis Folha cartografica).

In [ ]:
bounds = malha_cartog_1kk.bounds
bounds.head()

### Criando uma coluna region com os atributos retirados da funçao acima

In [ ]:
malha_cartog_1kk['region'] = [(left,right,bottom,top) for\
                               left,right,bottom,top in zip(bounds['minx'],bounds['maxx'],
                                                            bounds['miny'],bounds['maxy'])]

In [ ]:
# Indexando as folhas 1kk com função 'nomeador_grid()' descrita em './my_funcs.py' importado como 'mf'
df = pd.DataFrame(malha_cartog_1kk)
lista_grid = []
for index, row in df.iterrows():
    row['id_folha'] = (nomeador_grid(row.region[0],row.region[1],   
                                     row.region[3],row.region[2]))
    lista_grid.append(row.id_folha)

malha_cartog_1kk['id_folha'] = lista_grid
malha_cartog_1kk.drop(['id','left','right','bottom','top','region'],axis=1,inplace=True)

In [ ]:
malha_cartog_1kk.head(5)

In [ ]:
# Adicionando a camada vetorial ao bando de dados geopackage.
malha_cartog_1kk.to_file(gdb+'geodatabase.gpkg',driver='GPKG',layer='malha_cartog_1kk_wgs84')

In [ ]:
malha_cartog_1kk.head()

In [ ]:
malha_cartog_1kk['centroid'] = malha_cartog_1kk.centroid

In [ ]:
c = malha_cartog_1kk

In [ ]:
c['coords'] = c['geometry'].apply(lambda x: x.representative_point().coords[:])
c['coords'] = [coords[0] for coords in c['coords']]

In [ ]:
malha_cartog_1kk['coords'] = c['coords']

In [ ]:
plots.plot_brazil(malha_cartog_1kk.boundary)

for index, row in malha_cartog_1kk.iterrows():
    plt.annotate(s=row['id_folha'], xy=row['coords'],
                horizontalalignment='center')

In [ ]:
# Definindo Regions (W,E,S,N)
def regions(gdf):
    bounds = gdf.bounds 
    gdf['region'] = [(left,right,bottom,top) for left,right,bottom,top in zip(bounds['minx'],bounds['maxx'],
                                                                                bounds['miny'],bounds['maxy'])]

In [ ]:
# Definindo nomes a partir da articulação sistematica de folhas de cartas. Construindo uma lista e definindo como uma series.
def nomeador_malha(gdf):
    df = pd.DataFrame(gdf)
    lista_grid = []
    for index, row in df.iterrows():
        row['id_folha'] = (nomeador_grid(row.region[0],row.region[1],
                                              row.region[3],row.region[2],escala=5))
        lista_grid.append(row.id_folha)

    gdf['id_folha'] = lista_grid

In [ ]:
malha_cartog_25k = gpd.read_file(gdb+'cartografia/malha_cartog_25k_wgs84.shp')   # Escala de 1:25.000
malha_cartog_50k = gpd.read_file(gdb+'cartografia/malha_cartog_50k_wgs84.shp')   # Escala de 1:50.000
malha_cartog_100k = gpd.read_file(gdb+'cartografia/malha_cartog_100k_wgs84.shp') # Escala de 1:100.000

regions(malha_cartog_25k)
regions(malha_cartog_50k)
regions(malha_cartog_100k)

nomeador_malha(malha_cartog_25k)
nomeador_malha(malha_cartog_50k)
nomeador_malha(malha_cartog_100k)

# Percebe-se que há redundância no código, o que leva ao excesso de informação a ser escrita/lida e processada.

In [ ]:
malha_cartog_50k.drop(['id','left','right','bottom','top','region'],axis=1,inplace=True)
malha_cartog_25k.drop(['id','left','right','bottom','top','region'],axis=1,inplace=True)
malha_cartog_100k.drop(['id','left','right','bottom','top','region'],axis=1,inplace=True)

In [ ]:
malha_cartog_50k.to_file(gdb+'geodatabase.gpkg',driver='GPKG',layer='malha_cartog_50k_wgs84')
malha_cartog_25k.to_file(gdb+'geodatabase.gpkg',driver='GPKG',layer='malha_cartog_25k_wgs84')
malha_cartog_100k.to_file(gdb+'geodatabase.gpkg',driver='GPKG',layer='malha_cartog_100k_wgs84')

In [ ]:
malha_cartog_250k = gpd.read_file(gdb+'cartografia/malha_cartog_250k_wgs84.shp') # Escala de 1:250.000
regions(malha_cartog_250k)
nomeador_malha(malha_cartog_250k)
malha_cartog_250k.drop(['id','left','right','bottom','top','region'],axis=1,inplace=True)
malha_cartog_250k.to_file(gdb+'geodatabase.gpkg',driver='GPKG',layer='malha_cartog_250k_wgs84')

In [ ]:
malha_cartog_250k

### Construindo um selecionador de area a partir dos id das folhas cartograficas.

In [ ]:
def select_area(escala,id):
    malha_cartog = geometrias('malha_cartog_'+escala+'_wgs84')
    regions(malha_cartog)
    area = (malha_cartog[malha_cartog['id_folha'].str.contains(id)])
    return(area)

In [ ]:
def labels(gdf):
    gdf['coords'] = gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
    gdf['coords'] = [coords[0] for coords in gdf['coords']]

    for index, row in gdf.iterrows():
        plt.annotate(s=row['id_folha'], xy=row['coords'],horizontalalignment='center')

In [ ]:
area = select_area('25k','SC23_Z_A_IV')

In [ ]:
plot_base(area.boundary,atributo=None,camada='litologia_100k',mapa='Rio Paraim')
labels(area)

In [ ]:
socorro = select_area('250K','SF23_Y')

In [ ]:
socorro.boundary.plot()
labels(socorro)

In [ ]:
plot_base(socorro.boundary,atributo=None)